# HM3 CNN

## Download Data

In [5]:
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip
!unzip -q food-11.zip

Downloading...
From: https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
To: /content/food-11.zip
963MB [00:06, 138MB/s]


## Import Packages

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# 进度条
from tqdm.auto import tqdm

## Dataset, DataLoader, and Transforms

In [7]:
train_tfm = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),                     
])

test_tfm = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

In [8]:
batch_size = 128

train_set = DatasetFolder('food-11/training/labeled', loader=lambda x: Image.open(x), extensions='jpg', transform=train_tfm)
valid_set = DatasetFolder('food-11/validation', loader=lambda x: Image.open(x), extensions='jpg', transform=test_tfm)
unlabeled_set = DatasetFolder('food-11/training/unlabeled', loader=lambda x: Image.open(x), extensions='jpg', transform=train_tfm)
test_set = DatasetFolder('food-11/testing', loader=lambda x: Image.open(x), extensions='jpg', transform=test_tfm)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

## Model

In [11]:
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()

    self.cnn_layers = nn.Sequential(
        nn.Conv2d(3, 64, 3, 1, 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2, 2, 0),

        nn.Conv2d(64, 128, 3, 1, 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2, 2, 0),
        
        nn.Conv2d(128, 256, 3, 1, 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(4, 4, 0),
    )
    self.fc_layers = nn.Sequential(
        nn.Linear(256 * 8 * 8, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 11)
    )

    self.criterion = nn.CrossEntropyLoss()


  def forward(self, x):
    x = self.cnn_layers(x)

    x = x.flatten(1)

    x = self.fc_layers(x)
    return x

  def cal_loss(self, pred, y):
    return self.criterion(pred, y)

## Training

In [15]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

model = Classifier().to(device)
model.device = device

optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

n_epoch = 80

for epoch in range(n_epoch):
  model.train()

  train_loss = []
  train_accs = []

  for batch in tqdm(train_loader):
    imgs, labels = batch
    logits = model(imgs.to(device))
    loss = model.cal_loss(logits, labels.to(device))

    optimizer.zero_grad()

    loss.backward()

    grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

    optimizer.step()

    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

    train_loss.append(loss.item())
    train_accs.append(acc)

  train_l = sum(train_loss) / len(train_loss)
  train_acc = sum(train_accs) / len(train_accs)

  print(f"[ Train | {epoch + 1:03d}/{n_epoch:03d} ] loss = {train_l:.5f}, acc = {train_acc:.5f}")



[ Train | 001/080 ] loss = 2.28802, acc = 0.17719



[ Train | 002/080 ] loss = 1.90908, acc = 0.33844



[ Train | 003/080 ] loss = 1.76374, acc = 0.37000



[ Train | 004/080 ] loss = 1.64081, acc = 0.41125



[ Train | 005/080 ] loss = 1.48847, acc = 0.48750


...


[ Train | 076/080 ] loss = 0.00594, acc = 0.99969



[ Train | 077/080 ] loss = 0.03016, acc = 0.98937



[ Train | 078/080 ] loss = 0.00893, acc = 0.99969



[ Train | 079/080 ] loss = 0.00268, acc = 1.00000



[ Train | 080/080 ] loss = 0.00174, acc = 1.00000


## Testing

In [17]:
model.eval()

predictions = []

for batch in tqdm(test_loader):
  imgs, labels = batch
  with torch.no_grad():
    logits = model(imgs.to(device))

  predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [18]:
with open('predict.csv', 'w') as f:
  f.write('Id,Category\n')

  for i, pred in enumerate(predictions):
    f.write(f'{i},{pred}\n')